In [248]:
import sys
print("Python Version:", sys.version)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
pd.set_option('display.max_rows', 500)
%matplotlib inline

Python Version: 3.7.6 (default, Jan  8 2020, 20:23:39) [MSC v.1916 64 bit (AMD64)]


In [249]:
# yymmdd format
dates = ['190803',
        '190810',
        '190817',
        '190824',
        '190831',
        '190907',
        '190914',
        '190921',
        '190928']

# iterate through dates and pull in csv and cat dataframes together
df_turns = []
for date in dates:
    df_turns.append(pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_'+date+'.txt'))
    
# Concat all dataframes
df_turns = pd.concat(df_turns)  

In [250]:
#Clean up the column Names
new_col_names = [name.strip() for name in df_turns.columns]
df_turns.columns = new_col_names


In [251]:
# Remove audited rows
df_turns=df_turns[df_turns.DESC!='RECOVR AUD']
# remove column?
del df_turns['DESC']

In [252]:
# Create a DateTime column that can be used for timeseries
df_turns.loc[:,'DATETIME'] = pd.to_datetime(df_turns['DATE'] + ' ' + df_turns['TIME'])

#Convert Date column to proper date type.
df_turns['DATE']  = pd.to_datetime(df_turns['DATE'],format='%m/%d/%Y')

#convert time to datetime type
df_turns['TIME']=pd.to_datetime(df_turns['TIME'], format='%H:%M:%S')

In [253]:
#backup
df_backup=df_turns.copy()

In [254]:
#Sort in order of datetime
df_turns = df_turns.sort_values(by=['DATETIME']).reset_index()

In [255]:
# Caluclate Entries Aggregated at station level by DateTime
df_turns['Entry_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['ENTRIES'].transform(pd.Series.diff)['ENTRIES']

# Caluclate Exits Aggregated at station level by DateTime
df_turns['Exit_Diff']=df_turns.groupby(['STATION', 'C/A', 'UNIT', 'SCP'],as_index=False)['EXITS'].transform(pd.Series.diff)['EXITS']



In [256]:
#Absolute Value to deal with counting backward issues 
df_turns['Entry_Diff'] = abs(df_turns['Entry_Diff'])
df_turns['Exit_Diff']=abs(df_turns['Exit_Diff'])

# Calculate both
df_turns['Total_Traffic']=df_turns['Entry_Diff']+df_turns['Exit_Diff']

### Rami's Filtering

In [257]:
def Add_Weekday(data_frame, column='Date'):
    dmap = {0:'Mon', 1: 'Tue', 2: 'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
    #Convert column to date
    data_frame[column] = pd.to_datetime(data_frame[column])
    #Add column which shows the Weekday in integer
    data_frame['Day_Number'] = data_frame[column].apply(lambda x: x.dayofweek)
    #Add column which shows the Weekday in words
    data_frame['Weekday'] = data_frame['Day_Number'].map(dmap)
    return data_frame
df_turns_backup2 = df_turns.copy()

In [258]:
df_turns = Add_Weekday(df_turns, 'DATE')

In [259]:
#Filter out the Null Values
df_turns.dropna(subset=["Entry_Diff","Exit_Diff",'Total_Traffic'], axis=0, inplace=True)
df_turns.head()

index    C/A  UNIT       SCP         STATION LINENAME DIVISION  \
2554  118307  PTH03  R552  00-01-08  JOURNAL SQUARE        1      PTH   
2592  118308  PTH03  R552  00-01-08  JOURNAL SQUARE        1      PTH   
2636  118309  PTH03  R552  00-01-08  JOURNAL SQUARE        1      PTH   
2640  118310  PTH03  R552  00-01-08  JOURNAL SQUARE        1      PTH   
4489  118311  PTH03  R552  00-01-08  JOURNAL SQUARE        1      PTH   

           DATE                TIME  ENTRIES  EXITS            DATETIME  \
2554 2019-07-27 1900-01-01 00:24:57       65     30 2019-07-27 00:24:57   
2592 2019-07-27 1900-01-01 00:38:57       65     30 2019-07-27 00:38:57   
2636 2019-07-27 1900-01-01 00:52:57       65     30 2019-07-27 00:52:57   
2640 2019-07-27 1900-01-01 00:56:25     4043   3513 2019-07-27 00:56:25   
4489 2019-07-27 1900-01-01 01:06:57       65     30 2019-07-27 01:06:57   

      Entry_Diff  Exit_Diff  Total_Traffic  Day_Number Weekday  
2554         2.0        0.0            2.0           5     Sat  
2592         0.0        0.0            0.0           5     Sat  
2636         0.0        0.0            0.0           5     Sat  
2640      3978.0     3483.0         7461.0           5     Sat  
4489      3978.0     3483.0         7461.0           5     Sat

In [260]:
dfc_grouped = df_turns.groupby(['DATE', 'STATION', 'Weekday'])['Entry_Diff', 'Exit_Diff', 'Total_Traffic'].sum().reset_index()

C:\Users\atag3\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


In [261]:
#The upper_quant_df outputs a dataframe of the upper 95% values for each station
upper_quant_df = dfc_grouped.groupby(['STATION'])['Total_Traffic'].quantile([.95]).reset_index()
#Then I merge it with the grouped data 
df_turns_no_outlier = dfc_grouped.merge(upper_quant_df, on='STATION')
df_turns_no_outlier.head()

DATE STATION Weekday  Entry_Diff  Exit_Diff  Total_Traffic_x  level_1  \
0 2019-07-27    1 AV     Sat      4308.0     5146.0           9454.0     0.95   
1 2019-07-28    1 AV     Sun      4106.0     5309.0           9415.0     0.95   
2 2019-07-29    1 AV     Mon     14483.0    16525.0          31008.0     0.95   
3 2019-07-30    1 AV     Tue     16375.0    18206.0          34581.0     0.95   
4 2019-07-31    1 AV     Wed     16812.0    19777.0          36589.0     0.95   

   Total_Traffic_y  
0          40228.0  
1          40228.0  
2          40228.0  
3          40228.0  
4          40228.0

In [262]:
#delete the rows where The difference between Total_Traffic_y and Total_Traffic_x < 0
df_turns_no_outlier['Diff'] = df_turns_no_outlier['Total_Traffic_y'] - df_turns_no_outlier['Total_Traffic_x']
df_turns_no_outlier = df_turns_no_outlier[df_turns_no_outlier['Diff']>= 0]

In [263]:
df_turns_no_outlier.sample(50)

DATE          STATION Weekday  Entry_Diff  Exit_Diff  \
9798  2019-09-04     BOROUGH HALL     Wed     37462.0    25401.0   
12057 2019-08-27     CORTLANDT ST     Tue     20907.0    16498.0   
17545 2019-09-06  METS-WILLETS PT     Fri     15446.0    13973.0   
22946 2019-08-24    WEST FARMS SQ     Sat      4022.0     2976.0   
20918 2019-08-12  ROOSEVELT ISLND     Mon      7278.0     6806.0   
13601 2019-07-27    FLUSHING-MAIN     Sat     34218.0    29495.0   
12687 2019-08-27       DYCKMAN ST     Tue     12719.0     4009.0   
6061  2019-08-12            71 ST     Mon      5085.0     1649.0   
4787  2019-09-27       5 AV/53 ST     Fri     26714.0    29397.0   
4889  2019-09-03            5 AVE     Tue     12439.0    11200.0   
3658  2019-07-31  33 ST-RAWSON ST     Wed      9098.0     7483.0   
13455 2019-09-08     FAR ROCKAWAY     Sun      2290.0     2765.0   
16935 2019-09-26     LAFAYETTE AV     Thu      5522.0     2127.0   
20685 2019-08-31    RIT-ROOSEVELT     Sat      6499.0      102.0   
1556  2019-09-09           168 ST     Mon     19921.0     4811.0   
15951 2019-08-15  JAY ST-METROTEC     Thu     36482.0    25247.0   
20269 2019-09-25      PROSPECT AV     Wed     13254.0     7878.0   
16441 2019-08-04        KINGS HWY     Sun     10518.0     9863.0   
9577  2019-08-03    BEVERLEY ROAD     Sat      2047.0     2294.0   
21199 2019-09-10      SHEPHERD AV     Tue      3428.0     1867.0   
21444 2019-09-03        SPRING ST     Tue     21961.0    13037.0   
17836 2019-08-13      MORRIS PARK     Tue      2335.0     1955.0   
15124 2019-08-07          HIGH ST     Wed     10509.0    12033.0   
23544 2019-09-24    WTC-CORTLANDT     Tue     14537.0    11901.0   
6210  2019-09-04      72 ST-2 AVE     Wed     31608.0    27978.0   
737   2019-09-09  137 ST CITY COL     Mon     13835.0    12303.0   
449   2019-08-04  116 ST-COLUMBIA     Sun      4395.0     2471.0   
8860  2019-09-11     BEACH 105 ST     Wed       330.0      381.0   
19030 2019-08-10    NORTHERN BLVD     Sat      2664.0     2445.0   
11284 2019-08-10      CHAMBERS ST     Sat     20807.0    24567.0   
14736 2019-07-28     GROVE STREET     Sun      8255.0    11035.0   
17781 2019-08-21  MORISN AV/SNDVW     Wed      5881.0     5465.0   
14100 2019-09-23    FRESH POND RD     Mon      6504.0     6695.0   
14405 2019-09-13  GRAND ARMY PLAZ     Fri      7488.0     2777.0   
5957  2019-09-03            69 ST     Tue      4858.0     4832.0   
4739  2019-08-10       5 AV/53 ST     Sat       559.0      297.0   
22438 2019-08-20   V.CORTLANDT PK     Tue      7054.0     3011.0   
14830 2019-08-28      GUN HILL RD     Wed     10354.0    10351.0   
23150 2019-09-08      WHITLOCK AV     Sun       598.0      702.0   
3391  2019-09-17             3 AV     Tue      5567.0     5984.0   
19729 2019-08-20     PATH NEW WTC     Tue     61521.0    55582.0   
12164 2019-08-08   COURT SQ-23 ST     Thu      6366.0     2351.0   
14973 2019-09-14    HARLEM 148 ST     Sat      2236.0     2320.0   
18448 2019-09-27        NEREID AV     Fri      3773.0     1474.0   
11037 2019-08-15   CASTLE HILL AV     Thu      6065.0     5804.0   
7785  2019-09-07  ASTORIA DITMARS     Sat     10832.0    11060.0   
6603  2019-09-22             8 AV     Sun     24807.0    19774.0   
11366 2019-08-29      CHAUNCEY ST     Thu      3174.0     1556.0   
173   2019-09-12    103 ST-CORONA     Thu     21651.0    15714.0   
15610 2019-09-21     INTERVALE AV     Sat      2086.0     2470.0   

       Total_Traffic_x  level_1  Total_Traffic_y     Diff  
9798           62863.0     0.95          67595.7   4732.7  
12057          37405.0     0.95          41184.1   3779.1  
17545          29419.0     0.95          59616.5  30197.5  
22946           6998.0     0.95          12708.5   5710.5  
20918          14084.0     0.95          16047.5   1963.5  
13601          63713.0     0.95         116819.3  53106.3  
12687          16728.0     0.95          19080.4   2352.4  
6061            6734.0     0.95           7760.1   1026.1  
478

In [271]:
df_per_station = df_turns_no_outlier.groupby(['STATION']).mean()
df_per_station.sort_values('Total_Traffic_x', ascending=False).head(100)

Entry_Diff      Exit_Diff  Total_Traffic_x  level_1  \
STATION                                                                   
GRD CNTRL-42 ST  158716.627119  146812.033898    305528.661017     0.95   
34 ST-PENN STA   136887.271186  126923.932203    263811.203390     0.95   
34 ST-HERALD SQ  105549.067797  101689.559322    207238.627119     0.95   
TIMES SQ-42 ST    86884.677966   82042.779661    168927.457627     0.95   
23 ST             94877.779661   71002.203390    165879.983051     0.95   
14 ST-UNION SQ    84560.440678   80791.050847    165351.491525     0.95   
42 ST-PORT AUTH   89811.610169   74852.288136    164663.898305     0.95   
FULTON ST         83855.474576   75217.169492    159072.644068     0.95   
86 ST             69421.847458   66910.830508    136332.677966     0.95   
125 ST            66914.237288   57072.661017    123986.898305     0.95   
CANAL ST          66584.847458   53218.186441    119803.033898     0.95   
59 ST COLUMBUS    60556.067797   51648.576271    112204.644068     0.95   
47-50 STS ROCK    48284.254237   49949.525424     98233.779661     0.95   
59 ST             51811.135593   46092.694915     97903.830508     0.95   
FLUSHING-MAIN     51353.864407   45830.050847     97183.915254     0.95   
14 ST             51163.474576   45903.135593     97066.610169     0.95   
96 ST             50612.067797   41963.966102     92576.033898     0.95   
CHAMBERS ST       46039.983051   37449.881356     83489.864407     0.95   
PATH NEW WTC      43560.966102   39257.474576     82818.440678     0.95   
72 ST             40349.627119   39755.440678     80105.067797     0.95   
JKSN HT-ROOSVLT   43277.701754   33750.000000     77027.701754     0.95   
50 ST             41816.440678   33486.593220     75303.033898     0.95   
W 4 ST-WASH SQ    38242.796610   35973.949153     74216.745763     0.95   
42 ST-BRYANT PK   36285.881356   36202.694915     72488.576271     0.95   
28 ST             37509.016949   34094.237288     71603.254237     0.95   
ATL AV-BARCLAY    35700.067797   33934.711864     69634.779661     0.95   
LEXINGTON AV/53   32881.050847   33338.050847     66219.101695     0.95   
WALL ST           33564.949153   29166.644068     62731.593220     0.95   
7 AV              32354.932203   28918.711864     61273.644068     0.95   
145 ST            30305.152542   27755.932203     58061.084746     0.95   
JAY ST-METROTEC   32728.661017   23313.288136     56041.949153     0.95   
8 AV              31074.915254   24286.016949     55360.932203     0.95   
JAMAICA CENTER    27959.830508   26034.983051     53994.813559     0.95   
JOURNAL SQUARE    26426.830508   26938.169492     53365.000000     0.95   
57 ST-7 AV        29143.474576   22552.576271     51696.050847     0.95   
CHURCH AV         29264.186441   21683.389831     50947.576271     0.95   
GRAND ST          26971.728814   21204.288136     48176.016949     0.95   
BOWLING GREEN     24572.864407   23543.186441     48116.050847     0.95   
77 ST             25063.322034   22149.796610     47213.118644     0.95   
BOROUGH HALL      27583.677966   19189.457627     46773.135593     0.95   
72 ST-2 AVE       24826.372881   21921.745763     46748.118644     0.95   
B'WAY-LAFAYETTE   19639.067797   26719.322034     46358.389831     0.95   
THIRTY THIRD ST   27229.254237   18413.169492     45642.423729     0.95   
KINGS HWY         23024.881356   20776.389831     43801.271186     0.95   
DELANCEY/ESSEX    24257.491525   18958.237288     43215.728814     0.95   
116 ST            26503.593220   16394.271186     42897.864407     0.95   
103 ST            24331.050847   17910.881356     42241.932203     0.95   
33 ST             22964.203390   18231.779661     41195.983051     0.95   
5 AV/53 ST        19471.186441   21706.644068     41177.830508     0.95   
161/YANKEE STAD   25427.898305   15195.288136     40623.186441     0.95   
KEW GARDENS       20090.813559   20270.762712     40361.576271     0.95   
SUTPHIN-ARCHER    21068.423729   18493.237288

# End of group data cleaning 

## Begin Working on Demographics info

##### Rest of the team should use the above code so we are all working with the same data.

In [273]:
# bring in Census data
cen_dat = pd.read_csv('geocoded_cen_dat.csv')
cen_vars=['STATION',
          'LAT',
          'LON',
          'p_f_pop',
          'p_emp_info',
          'p_emp_prof',
          'p_75_100k',
          'p_over_100k']

df_per_station = pd.merge(df_per_station, cen_dat[cen_vars],  how='left', left_on=['STATION'], right_on = ['STATION'])
df_per_station =df_per_station.drop_duplicates(['STATION'])
df_per_station.sort_values('Total_Traffic_x', ascending=False).head(100)

STATION     Entry_Diff      Exit_Diff  Total_Traffic_x  level_1  \
290  GRD CNTRL-42 ST  158716.627119  146812.033898    305528.661017     0.95   
81    34 ST-PENN STA  136887.271186  126923.932203    263811.203390     0.95   
78   34 ST-HERALD SQ  105549.067797  101689.559322    207238.627119     0.95   
420   TIMES SQ-42 ST   86884.677966   82042.779661    168927.457627     0.95   
62             23 ST   94877.779661   71002.203390    165879.983051     0.95   
22    14 ST-UNION SQ   84560.440678   80791.050847    165351.491525     0.95   
90   42 ST-PORT AUTH   89811.610169   74852.288136    164663.898305     0.95   
280        FULTON ST   83855.474576   75217.169492    159072.644068     0.95   
140            86 ST   69421.847458   66910.830508    136332.677966     0.95   
14            125 ST   66914.237288   57072.661017    123986.898305     0.95   
212         CANAL ST   66584.847458   53218.186441    119803.033898     0.95   
111   59 ST COLUMBUS   60556.067797   51648.576271    112204.644068     0.95   
94    47-50 STS ROCK   48284.254237   49949.525424     98233.779661     0.95   
109            59 ST   51811.135593   46092.694915     97903.830508     0.95   
268    FLUSHING-MAIN   51353.864407   45830.050847     97183.915254     0.95   
20             14 ST   51163.474576   45903.135593     97066.610169     0.95   
147            96 ST   50612.067797   41963.966102     92576.033898     0.95   
222      CHAMBERS ST   46039.983051   37449.881356     83489.864407     0.95   
378     PATH NEW WTC   43560.966102   39257.474576     82818.440678     0.95   
124            72 ST   40349.627119   39755.440678     80105.067797     0.95   
315  JKSN HT-ROOSVLT   43277.701754   33750.000000     77027.701754     0.95   
100            50 ST   41816.440678   33486.593220     75303.033898     0.95   
432   W 4 ST-WASH SQ   38242.796610   35973.949153     74216.745763     0.95   
89   42 ST-BRYANT PK   36285.881356   36202.694915     72488.576271     0.95   
70             28 ST   37509.016949   34094.237288     71603.254237     0.95   
157   ATL AV-BARCLAY   35700.067797   33934.711864     69634.779661     0.95   
330  LEXINGTON AV/53   32881.050847   33338.050847     66219.101695     0.95   
435          WALL ST   33564.949153   29166.644068     62731.593220     0.95   
121             7 AV   32354.932203   28918.711864     61273.644068     0.95   
24            145 ST   30305.152542   27755.932203     58061.084746     0.95   
311  JAY ST-METROTEC   32728.661017   23313.288136     56041.949153     0.95   
134             8 AV   31074.915254   24286.016949     55360.932203     0.95   
309   JAMAICA CENTER   27959.830508   26034.983051     53994.813559     0.95   
316   JOURNAL SQUARE   26426.830508   26938.169492     53365.000000     0.95   
108       57 ST-7 AV   29143.474576   22552.576271     51696.050847     0.95   
228        CHURCH AV   29264.186441   21683.389831     50947.576271     0.95   
286         GRAND ST   26971.728814   21204.288136     48176.016949     0.95   
197    BOWLING GREEN   24572.864407   23543.186441     48116.050847     0.95   
130            77 ST   25063.322034   22149.796610     47213.118644     0.95   
193     BOROUGH HALL   27583.677966   19189.457627     46773.135593     0.95   
126      72 ST-2 AVE   24826.372881   21921.745763     46748.118644     0.95   
169  B'WAY-LAFAYETTE   19639.067797   26719.322034     46358.389831     0.95   
419  THIRTY THIRD ST   27229.254237   18413.169492     45642.423729     0.95   
320        KINGS HWY   23024.881356   20776.389831     43801.271186     0.95   
248   DELANCEY/ESSEX   24257.491525   18958.237288     43215.728814     0.95   
10            116 ST   26503.593220   16394.271186     42897.864407     0.95   
1             103 ST   24331.050847   17910.881356     42241.932203     0.95   
76             33 ST   22964.203390   18231.779661     41195.983051     0.95   
97        5 AV/53 ST   19471.186441   21706.644068     41177.830508     0.95   
31   161/YANK

In [247]:
cen_dat.head()

STATION DIVISION          all        LAT        LON       geo_type  \
0           1 AV      BMT  19409616052  40.731327 -73.982435  establishment   
1         103 ST      IND   2344428272  40.796286 -73.961648  establishment   
2         103 ST      IRT   4382729249  40.796286 -73.961648  establishment   
3  103 ST-CORONA      IRT   2641087500  40.749827 -73.862746  establishment   
4         104 ST      BMT     18924221  40.695146 -73.844305  establishment   

   State_FIPS  County_FIPS  Tract_FIPS   f_pop  all_pop  p_emp_info   p_f_pop  \
0          36           61        4400  9661.0  16684.0    0.077069  0.579058   
1          36           61       18900  6869.0  11800.0    0.057668  0.582119   
2          36           61       18900  6869.0  11800.0    0.057668  0.582119   
3          36           81       40501     NaN      NaN         NaN       NaN   
4          36           81        2800  1602.0   3382.0    0.033613  0.473684   

   p_emp_prof  p_75_100k  p_over_100k  
0    0.226378      0.166        0.396  
1    0.170079      0.050        0.235  
2    0.170079      0.050        0.235  
3         NaN        NaN          NaN  
4    0.039016      0.077        0.080

In [285]:
df_per_station['Traffic_Rank'] = df_per_station['Total_Traffic_x'].rank(ascending=True,pct=True)*100
df_per_station['p_f_pop_Rank'] = df_per_station['p_f_pop_y'].rank(ascending=True,pct=True)*100
df_per_station['p_emp_info_Rank'] = df_per_station['p_emp_info_y'].rank(ascending=True,pct=True)*100
df_per_station['p_emp_prof_Rank'] = df_per_station['p_emp_prof_y'].rank(ascending=True,pct=True)*100
df_per_station['p_75_100k_Rank'] = df_per_station['p_75_100k_y'].rank(ascending=True,pct=True)*100
df_per_station['p_over_100k_Rank'] = df_per_station['p_over_100k_y'].rank(ascending=True,pct=True)*100

df_per_station.sort_values('Total_Traffic_x', ascending=False).head(100)

df_per_station['Recommending_Score'] = (df_per_station['Traffic_Rank'] * .40) +\
                                       (df_per_station['p_f_pop_Rank'] * .20)+ \
                                       (df_per_station['p_emp_info_Rank'] * .15)+ \
                                       (df_per_station['p_emp_prof_Rank'] * .15)+ \
                                       (df_per_station['p_75_100k_Rank'] * .05)+ \
                                       (df_per_station['p_over_100k_Rank'] * .05)

df_per_station.sort_values('Recommending_Score', ascending=False).head(100)

STATION     Entry_Diff      Exit_Diff  Total_Traffic_x  level_1  \
435          WALL ST   33564.949153   29166.644068     62731.593220     0.95   
70             28 ST   37509.016949   34094.237288     71603.254237     0.95   
103            51 ST   17204.596491   19178.929825     36383.526316     0.95   
280        FULTON ST   83855.474576   75217.169492    159072.644068     0.95   
62             23 ST   94877.779661   71002.203390    165879.983051     0.95   
130            77 ST   25063.322034   22149.796610     47213.118644     0.95   
157   ATL AV-BARCLAY   35700.067797   33934.711864     69634.779661     0.95   
272  FOREST HILLS 71   21933.508475   11393.271186     33326.779661     0.95   
197    BOWLING GREEN   24572.864407   23543.186441     48116.050847     0.95   
0               1 AV   12912.457627   14543.423729     27455.881356     0.95   
117    66 ST-LINCOLN   16086.237288   15528.406780     31614.644068     0.95   
419  THIRTY THIRD ST   27229.254237   18413.169492     45642.423729     0.95   
331  LEXINGTON AV/63   17276.338983   18571.101695     35847.440678     0.95   
119   68ST-HUNTER CO   17417.203390   15076.627119     32493.830508     0.95   
273      FRANKLIN AV   18204.728814   12709.000000     30913.728814     0.95   
22    14 ST-UNION SQ   84560.440678   80791.050847    165351.491525     0.95   
239     CORTLANDT ST   16855.745763   12757.694915     29613.440678     0.95   
121             7 AV   32354.932203   28918.711864     61273.644068     0.95   
89   42 ST-BRYANT PK   36285.881356   36202.694915     72488.576271     0.95   
319      KEW GARDENS   20090.813559   20270.762712     40361.576271     0.95   
1             103 ST   24331.050847   17910.881356     42241.932203     0.95   
155         ASTOR PL   13582.830508   15182.915254     28765.745763     0.95   
169  B'WAY-LAFAYETTE   19639.067797   26719.322034     46358.389831     0.95   
76             33 ST   22964.203390   18231.779661     41195.983051     0.95   
147            96 ST   50612.067797   41963.966102     92576.033898     0.95   
408        SPRING ST   21565.440678   13451.338983     35016.779661     0.95   
108       57 ST-7 AV   29143.474576   22552.576271     51696.050847     0.95   
432   W 4 ST-WASH SQ   38242.796610   35973.949153     74216.745763     0.95   
247        DEKALB AV   26264.661017    9574.288136     35838.949153     0.95   
290  GRD CNTRL-42 ST  158716.627119  146812.033898    305528.661017     0.95   
188        BERGEN ST   12478.898305    8333.288136     20812.186441     0.95   
420   TIMES SQ-42 ST   86884.677966   82042.779661    168927.457627     0.95   
81    34 ST-PENN STA  136887.271186  126923.932203    263811.203390     0.95   
78   34 ST-HERALD SQ  105549.067797  101689.559322    207238.627119     0.95   
212         CANAL ST   66584.847458   53218.186441    119803.033898     0.95   
207  BROOKLYN BRIDGE   16250.474576   13477.084746     29727.559322     0.95   
20             14 ST   51163.474576   45903.135593     97066.610169     0.95   
111   59 ST COLUMBUS   60556.067797   51648.576271    112204.644068     0.95   
184       BEDFORD AV   20550.966102   12695.508475     33246.474576     0.95   
446    WTC-CORTLANDT   11159.508475    9210.576271     20370.084746     0.95   
134             8 AV   31074.915254   24286.016949     55360.932203     0.95   
140            86 ST   69421.847458   66910.830508    136332.677966     0.95   
418        THIRTY ST    9626.813559    7376.186441     17003.000000     0.95   
137     81 ST-MUSEUM   12412.033898   10248.728814     22660.762712     0.95   
392        RECTOR ST   12317.457627    6323.474576     18640.932203     0.95   
135         8 ST-NYU   14247.101695    9563.169492     23810.271186     0.95   
100            50 ST   41816.440678   33486.593220     75303.033898     0.95   
98        5 AV/59 ST   13561.423729    9859.271186     23420.694915     0.95   
320        KINGS HWY   23024.881356   20776.389831     43801.271186     0.95   
311  JAY ST-M

In [176]:
import folium
from folium.plugins import MarkerCluster

In [244]:
#reset index of df per station
df_per_station = df_per_station.sort_values('Total_Traffic', ascending=False).head(100).reset_index(drop=True)

#stations
stations = df_per_station[['LAT','LON']]
stations_list=stations.values.tolist()
#Define coordinates of map center
nyc_coords = [40.7528, -73.9560]

#Create the map
my_map = folium.Map(location = nyc_coords, zoom_start = 12)

#Display the map
my_map

for point in range(0, len(stations_list)):
    if point <=10:
        folium.Marker(stations_list[point], popup=df_per_station.STATION[point]).add_to(my_map)
    else:
        folium.Circle(stations_list[point], popup=df_per_station.STATION[point],fill=True,radius=100,color='red').add_to(my_map)
    
my_map

AttributeError: 'DataFrame' object has no attribute 'STATIOJN'